# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("city.csv")
df.head()

,Unnamed: 0,Temperature,Latitude,Longitude,Humidity,Wind Speed,Cloud,Country,Date,City
0,0,-22.76,71.9667,102.5000,98,3.65,39,RU,1637192390,khatanga
1,1,69.87,-24.8667,113.6333,64,11.50,75,AU,1637192390,carnarvon
2,2,59.92,-33.9258,18.4232,58,17.27,40,ZA,1637192200,cape town
3,3,76.37,-20.0064,57.6608,70,4.00,13,MU,1637192391,grand gaube
4,4,59.29,-33.9180,25.5701,94,16.11,90,ZA,1637192290,port elizabeth


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations=df[["Latitude", "Longitude"]]

weight=df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
Weather_df=df[(df["Temperature"] <=70) & (df["Humidity"]<=80)] 
Weather_df=Weather_df.dropna()
Weather_df

,Unnamed: 0,Temperature,Latitude,Longitude,Humidity,Wind Speed,Cloud,Country,Date,City
1,1,69.87,-24.8667,113.6333,64,11.50,75,AU,1637192390,carnarvon
2,2,59.92,-33.9258,18.4232,58,17.27,40,ZA,1637192200,cape town
6,6,57.92,-46.2817,169.8464,61,4.88,28,NZ,1637192391,kaitangata
9,9,18.91,57.7900,-152.4072,58,21.85,90,US,1637192392,kodiak
12,12,17.02,53.4500,108.9833,76,2.26,33,RU,1637192393,ust-barguzin
...,...,...,...,...,...,...,...,...,...,...
560,560,51.96,35.1628,112.8650,50,3.98,0,CN,1637192543,xixiang
563,563,15.62,49.6334,-105.9845,73,21.85,42,CA,1637192544,assiniboia
564,564,62.04,-38.5473,-58.7368,51,10.76,21,AR,1637192545,necochea
575,575,64.15,23.4333,87.2833,54,4.72,66,IN,1637192548,barjora


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=Weather_df[["City", "Country", "Latitude","Longitude"]]
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-6-62db1cbbf50b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Latitude,Longitude,Hotel Name
1,carnarvon,AU,-24.8667,113.6333,
2,cape town,ZA,-33.9258,18.4232,
6,kaitangata,NZ,-46.2817,169.8464,
9,kodiak,US,57.7900,-152.4072,
12,ust-barguzin,RU,53.4500,108.9833,
...,...,...,...,...,...
560,xixiang,CN,35.1628,112.8650,
563,assiniboia,CA,49.6334,-105.9845,
564,necochea,AR,-38.5473,-58.7368,
575,barjora,IN,23.4333,87.2833,


In [5]:

hotel_df = Weather_df

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

   
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")



Retrieving Results for Index 1: carnarvon.
Closest hotel is Carnarvon.
------------
Retrieving Results for Index 2: cape town.
Closest hotel is Cape Town.
------------
Retrieving Results for Index 6: kaitangata.
Closest hotel is Kaitangata.
------------
Retrieving Results for Index 9: kodiak.
Closest hotel is Kodiak.
------------
Retrieving Results for Index 12: ust-barguzin.
Closest hotel is Ust-Barguzin.
------------
Retrieving Results for Index 14: avera.
Closest hotel is Avera.
------------
Retrieving Results for Index 15: east london.
Closest hotel is East London.
------------
Retrieving Results for Index 16: bluff.
Closest hotel is Bluff.
------------
Retrieving Results for Index 21: new norfolk.
Closest hotel is New Norfolk.
------------
Retrieving Results for Index 25: hermanus.
Closest hotel is Hermanus.
------------
Retrieving Results for Index 26: chuy.


KeyboardInterrupt: 

In [6]:
hotel_df

,Unnamed: 0,Temperature,Latitude,Longitude,Humidity,Wind Speed,Cloud,Country,Date,City,Hotel Name
1,1,69.87,-24.8667,113.6333,64,11.50,75,AU,1637192390,carnarvon,Carnarvon
2,2,59.92,-33.9258,18.4232,58,17.27,40,ZA,1637192200,cape town,Cape Town
6,6,57.92,-46.2817,169.8464,61,4.88,28,NZ,1637192391,kaitangata,Kaitangata
9,9,18.91,57.7900,-152.4072,58,21.85,90,US,1637192392,kodiak,Kodiak
12,12,17.02,53.4500,108.9833,76,2.26,33,RU,1637192393,ust-barguzin,Ust-Barguzin
...,...,...,...,...,...,...,...,...,...,...,...
560,560,51.96,35.1628,112.8650,50,3.98,0,CN,1637192543,xixiang,NaN
563,563,15.62,49.6334,-105.9845,73,21.85,42,CA,1637192544,assiniboia,NaN
564,564,62.04,-38.5473,-58.7368,51,10.76,21,AR,1637192545,necochea,NaN
575,575,64.15,23.4333,87.2833,54,4.72,66,IN,1637192548,barjora,NaN


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

plt.savefig("images")

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>